In [3]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin
db_pomoci = pd.read_csv("db_pomoci.csv")

In [3]:
df_scraped = pd.read_csv("data/df_scraped.csv")
df_scraped.head()

,Unnamed: 0,Website,Phone Numbers
0,0.0,http://www.pdz.cz,"121 407 9329, 64 24 22 18, 420 466 400 116"
1,1.0,http://www.medsix.cz/?option=com_content&view=...,"483 341 232, 483 300 087, 483 310 919, 483 341..."
2,2.0,https://www.osbonanza.cz,NaN
3,3.0,https://www.svetlokadanzs.cz,NaN
4,4.0,https://jindrichuvhradec.charita.cz/sluzba/852/,"731 402 982, 731 604 554, 605 849 877"


In [10]:
df_res = df_scraped.rename(columns={'web': 'web', 'Telefon': 'Telefon'}).drop("Unnamed: 0", axis= 1)
df_res = df_res.rename(columns= {"Website":"web"})
df_res = df_res.rename(columns= {"Phone Numbers":"phones_scraped"})
df_res['phones_scraped'] = df_res['phones_scraped'].str.split(', ')
df_res_exp = df_res.explode('phones_scraped').reset_index(drop=True)
db_pomoci = db_pomoci[['E-mail', 'Telefon']]
db_pomoci['Telefon'] = db_pomoci['Telefon'].str.split(', ')
db_pomoci_exp = db_pomoci.explode('Telefon').reset_index(drop=True)
db_pomoci_exp.rename(columns = {"E-mail":"web"}, inplace=True)
db_pomoci_exp['table'] = 'initial'
df_res_exp['table'] = 'scraped'
df_res_exp


,web,phones_scraped
0,http://www.pdz.cz,"[121 407 9329, 64 24 22 18, 420 466 400 116]"
1,http://www.medsix.cz/?option=com_content&view=...,"[483 341 232, 483 300 087, 483 310 919, 483 34..."
2,https://www.osbonanza.cz,NaN
3,https://www.svetlokadanzs.cz,NaN
4,https://jindrichuvhradec.charita.cz/sluzba/852/,"[731 402 982, 731 604 554, 605 849 877]"


In [15]:
df_res_exp.rename(columns= {"phones_scraped":"Telefon"}, inplace=True)

In [16]:
df_union = pd.concat([db_pomoci_exp, df_res_exp])

In [15]:
df_union

,web,Telefon,table
0,pdz-hk@pdz.cz,778 526 216,initial
1,fokusopava@seznam.cz,553 652 433,initial
2,fokusopava@seznam.cz,723 457 998,initial
3,pdz-cr@pdz.cz,469 626 383,initial
4,pdz-cr@pdz.cz,774 915 755,initial
...,...,...,...
4969,http://www.psychoterapeuti.cz/adresar-psychote...,222 531 008,scraped
4970,http://www.psychoterapeuti.cz/adresar-psychote...,724 614 509,scraped
4971,http://www.psychoterapeuti.cz/adresar-psychote...,261 105 410,scraped
4972,http://www.psychoterapeuti.cz/adresar-psychote...,222 324 277,scraped


# Searching for empty enrties 

In [34]:
import duckdb

# phones_frequency = duckdb.sql("""
# SELECT
#     DISTINCT web,
#     Telefon,
#     COUNT(Telefon) AS count_of_phones
# FROM
#     df_union
# GROUP BY
#     web, Telefon
# ORDER BY
#     count_of_phones DESC
# """
# ).df()
phones_frequency = duckdb.sql("""
WITH RankedPhones AS (
    SELECT
        web,
        Telefon,
        COUNT(Telefon) AS count_of_phones,
        ROW_NUMBER() OVER (PARTITION BY web ORDER BY COUNT(Telefon) DESC) AS rn
    FROM
        df_union
    GROUP BY
        web, Telefon
)
SELECT
    web,
    Telefon,
    count_of_phones
FROM
    RankedPhones
WHERE
    rn = 1
ORDER BY
    count_of_phones DESC
"""
).df()
phones_frequency

,web,Telefon,count_of_phones
0,info@klinikaeset.cz,272 937 500,7
1,None,533 302 324,6
2,info@psychologove.cz,466 611 797,5
3,pravni.poradenstvi@krumlov.cz,774 110 124,4
4,hradcanska@kappa-praha.cz,730 513 343,4
...,...,...,...
1821,https://www.lexik.cz/pedagogicko-psychologicka...,None,0
1822,https://www.psychiatrie-hajda.cz/#utm_source=f...,None,0
1823,https://www.zavinac.sdb.cz,None,0
1824,mb-psychoterapie.cz,None,0


In [36]:
not_matched = phones_frequency[phones_frequency["Telefon"].isna()]

In [37]:
not_matched.to_csv("data/not_matched.csv")

# Validating missing 

In [44]:
link= "http://cross.prevent99.cz"



<a href="https://www.prevent99.cz/kontakty/">KONTAKTY</a>
https://www.prevent99.cz/kontakty/
<a href="https://www.prevent99.cz/kontakty/">Kontakty</a>
https://www.prevent99.cz/kontakty/


In [40]:

# Regular expressions for matching emails and phone numbers
email_regex = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
phone_regex = re.compile(r"\+?(\d[\d\-\(\) ]{9,}\d)")

# List to collect data
data = []

# Function to scrape a single URL
def scrape_website_main_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        return emails, phones
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return set(), set()

def scrape_website_contacts(base_url, path='', visited_urls=set()):
    full_url = urljoin(base_url, path)
    if full_url in visited_urls:
        return set(), set()  # Return empty sets if URL has already been visited
    visited_urls.add(full_url)

    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        # Checking for links that may contain contact info
        contact_links = []
        for link in soup.find_all('a', href=True):
            if 'kontakt' in link.text.lower() or 'contact' in link.text.lower() or 'kontakty' in link.text.lower():
                contact_href = urljoin(full_url, link['href'])
                if contact_href not in visited_urls:
                    contact_links.append(contact_href)

        # Scrape each potential contact page
        for link in contact_links:
            sub_emails, sub_phones = scrape_website_contacts(base_url, link, visited_urls)
            emails.update(sub_emails)
            phones.update(sub_phones)

        return emails, phones
    except requests.RequestException as e:
        print(f"Error fetching {full_url}: {e}")
        return set(), set()


visited_urls = set()
emails, phones = scrape_website_main_page(link)
print(emails)
print("")
print(phones)
if  ('' in emails) or  ('' in phones):        
    emails, phones = scrape_website_contacts(link, visited_urls=visited_urls)
print(emails)
print("")
print(phones)


{'marie.novakova@prevent99.cz', 'cross@prevent99.czT', 'frydl@prevent99.cz', 'sosnova@prevent99.cz', 'kreslova@prevent99.cz'}

set()
{'marie.novakova@prevent99.cz', 'cross@prevent99.czT', 'frydl@prevent99.cz', 'sosnova@prevent99.cz', 'kreslova@prevent99.cz'}

set()


In [46]:
website_df = df_union[df_union['web'] == "mdrdova@spzkolobezka.cz"]
website_df['Telefon']#.mode()[0]

553    NaN
Name: Telefon, dtype: object

In [51]:
link_main = ["http://klubtulip.cz"]
for url in tqdm(link_main):
    #print(f"Scraping {url}")
    
    emails, phones = scrape_website_main_page(url)
    if  ('' in emails) or  ('' in phones):        
        emails, phones = scrape_website_contacts(url, visited_urls=visited_urls)
    
emails, phones 


100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


({'tulip@uhradiste.charita.cz'}, {'606 672 239', '725 520 998'})

In [48]:
link_main = "http://klubtulip.cz"

response = requests.get(link_main)
soup = BeautifulSoup(response.text, 'html.parser')
visited_urls = set()
contact_links = []
for link in soup.find_all('a', href=True):
    if "kontakty" in link.text.lower():
        print(link)
        contact_href = urljoin(link_main, link['href'])
        if contact_href not in visited_urls:
                    contact_links.append(contact_href)
        print(contact_href)
        for link in contact_links:
            sub_emails, sub_phones = scrape_website_main_page(link)
            emails.update(sub_emails)
            phones.update(sub_phones)
print(emails)
print("")
print(phones)

{'antonu@prevent99.cz', 'nrp@prevent99.cz', 'stastka@prevent99.cz', 'kccb@prevent99.czKontaktn', 'frydl@prevent99.cz', 'sas@prevent99.czCross', 'bauerova@prevent99.czV', 'kcpt@prevent99.cz', 'kreslova@prevent99.cz', 'kancelar@prevent99.czTel.', 'nemcova@prevent99.cz', 'kcpt@prevent99.czKontaktn', 'kcst@prevent99.czPoradna', 'harant@prevent99.cz', 'cadova@prevent99.cz.', 'spackova@prevent99.cz', 'kcjh@prevent99.cz', 'ap.strakonice@prevent99.czAdiktologick', 'platzerova@prevent99.cz', 'cadova@prevent99.cz', 'vacikova@prevent99.czV', 'dc@prevent99.czAdiktologick', 'velkova@prevent99.cz', 'kccb@prevent99.cz', 'kcst@prevent99.czWeb', 'marie.novakova@prevent99.cz', 'cross@prevent99.czT', 'cross@prevent99.czStation', 'nrp@prevent99.czPodpora', 'sosnova@prevent99.cz', 'nemec@prevent99.cz', 'binderova@prevent99.czInfo', 'klajn@prevent99.cz', 'ap.prachatice@prevent99.czSubstitu', 'spot@prevent99.cz', 'podzimek@prevent99.cz', 'sc@prevent99.czDol', 'kcjh@prevent99.czKontaktn', 'asv@prevent99.czKon

In [56]:
phones_frequency_1 = duckdb.sql("""
    SELECT
        web,
        Telefon
    FROM
        df_union
    WHERE web in 
    (SELECT DISTINCT web from not_matched)
"""
).df()
phones_frequency_1

,web,Telefon
0,http://www.cheiront.cz/,None
1,https://www.firmy.cz/detail/12844251-egomed-ja...,None
2,https://www.ocss.cz,None
3,http://www.zdar.charita.cz,None
4,https://www.facebook.com/konzultarnavsetin,None
...,...,...
317,https://www.romanojasnica.webnode.cz,None
318,https://www.jiskraduse.cz,None
319,https://www.psychiatrietrebic.cz,None
320,https://pisek.charita.cz/jak-pomahame/oblastni...,None


# Validating data

In [6]:
df_scraped_not_matched = pd.read_csv("data/df_scraped_not_matched.csv")
df_scraped = pd.read_csv("data/df_scraped.csv")

In [7]:
df_scraped_not_matched.shape

(321, 3)

In [9]:
df_scraped.head()

,Unnamed: 0,Website,Phone Numbers
0,0.0,http://www.pdz.cz,"121 407 9329, 64 24 22 18, 420 466 400 116"
1,1.0,http://www.medsix.cz/?option=com_content&view=...,"483 341 232, 483 300 087, 483 310 919, 483 341..."
2,2.0,https://www.osbonanza.cz,NaN
3,3.0,https://www.svetlokadanzs.cz,NaN
4,4.0,https://jindrichuvhradec.charita.cz/sluzba/852/,"731 402 982, 731 604 554, 605 849 877"


In [11]:
df_res_exp

,web,phones_scraped
0,http://www.pdz.cz,121 407 9329
1,http://www.pdz.cz,64 24 22 18
2,http://www.pdz.cz,420 466 400 116
3,http://www.medsix.cz/?option=com_content&view=...,483 341 232
4,http://www.medsix.cz/?option=com_content&view=...,483 300 087
...,...,...
4969,http://www.psychoterapeuti.cz/adresar-psychote...,222 531 008
4970,http://www.psychoterapeuti.cz/adresar-psychote...,724 614 509
4971,http://www.psychoterapeuti.cz/adresar-psychote...,261 105 410
4972,http://www.psychoterapeuti.cz/adresar-psychote...,222 324 277


In [18]:
df_union

,web,Telefon,table
0,pdz-hk@pdz.cz,778 526 216,initial
1,fokusopava@seznam.cz,553 652 433,initial
2,fokusopava@seznam.cz,723 457 998,initial
3,pdz-cr@pdz.cz,469 626 383,initial
4,pdz-cr@pdz.cz,774 915 755,initial
...,...,...,...
4969,http://www.psychoterapeuti.cz/adresar-psychote...,222 531 008,scraped
4970,http://www.psychoterapeuti.cz/adresar-psychote...,724 614 509,scraped
4971,http://www.psychoterapeuti.cz/adresar-psychote...,261 105 410,scraped
4972,http://www.psychoterapeuti.cz/adresar-psychote...,222 324 277,scraped


In [33]:
df_union.groupby(["web","Telefon"]).count().sort_values(ascending = False, by = "table")

table
web                                                Telefon           
info@klinikaeset.cz                                272 937 500      7
info@psychologove.cz                               734 502 526      5
                                                   466 611 797      5
pravni.poradenstvi@krumlov.cz                      774 110 124      4
hradcanska@kappa-praha.cz                          730 513 343      4
...                                                               ...
http://www.psychoterapeuti.cz/adresar-psychoter... 775 182 608      1
                                                   774 979 714      1
                                                   774 918 150      1
                                                   770 623 916      1
zvonicekjosef@seznam.cz                            577 923 390      1

[5785 rows x 1 columns]

In [35]:
weby = set(df_union["web"])
len(weby)

1826

In [64]:
for web in weby:
    try:
        print(web)
        filtered_df = df_union[df_union["web"] == web]
        r_df = filtered_df.groupby(["web","Telefon"]).count().sort_values(ascending = False, by = "table")
        print(r_df.iloc[0])
    except :
        pass
    


renata.kulkova@tiscali.cz
table    1
Name: (renata.kulkova@tiscali.cz, 774 635 690), dtype: int64
https://www.sos-vesnicky.cz
table    1
Name: (https://www.sos-vesnicky.cz, 420 233 335 452), dtype: int64
blankastefanovicova@seznam.cz
table    1
Name: (blankastefanovicova@seznam.cz, 352 600 347), dtype: int64
https://www.jekhetane.cz
klinickapsychologieR@klinic.cz
table    1
Name: (klinickapsychologieR@klinic.cz, 603 396 912), dtype: int64
https://kbterapie.webnode.cz/
http://www.sasmcb.cz
table    1
Name: (http://www.sasmcb.cz, 420 910 441 242), dtype: int64
jkanerova@volny.cz
table    1
Name: (jkanerova@volny.cz, 736 200 687), dtype: int64
http://www.klinic.cz/
table    1
Name: (http://www.klinic.cz/, 420 603 396 912), dtype: int64
https://www.firmy.cz/detail/1355618-mgr-iva-matejckova-zezulova-tachov.html
table    1
Name: (https://www.firmy.cz/detail/1355618-mgr-iva-matejckova-zezulova-tachov.html, 420 732 110 025), dtype: int64
https://mudr-jitka-bartkova.modernilekar.cz/
table    1

In [63]:
df_union[df_union["web"] == "https://www.jekhetane.cz"]

,web,Telefon,table
1296,https://www.jekhetane.cz,NaN,scraped
